In [236]:
import torch
import numpy as np
import pandas as pd
import os
import sys

In [237]:
from sklearn.metrics import classification_report

In [238]:
from torch.utils.data import TensorDataset, DataLoader

In [239]:
import torch.nn as nn
from torch import optim

In [240]:
from sklearn.model_selection import train_test_split

In [241]:
from sklearn.preprocessing import StandardScaler

### Baseline Split

In [99]:
data = pd.read_csv('../data/gesture-recognition-and-biometrics-electromyogram-grabmyo-1.0.2/features_split.csv')

In [100]:
data.head()

,Unnamed: 0,session,participant,gesture,index,iemg,mav,ssi,myopulse,wflen,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,0,1,1,10,0,11996.863087,0.041842,997.135823,6.224512,8143.485828,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,1,1,1,10,0,10550.150337,0.036796,1019.627142,6.443555,6646.721181,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,2,1,1,10,0,9230.366582,0.032193,630.013233,6.033789,6023.438546,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,3,1,1,10,0,15949.504704,0.055627,1995.234560,4.999609,9318.559036,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,4,1,1,10,0,15936.956936,0.055584,1830.412922,5.870117,10476.465579,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713


In [101]:
data.columns

Index(['Unnamed: 0', 'session', 'participant', 'gesture', 'index', 'iemg',
       'mav', 'ssi', 'myopulse', 'wflen',
       ...
       'kurtosis_f_w3', 'kurtosis_f_w4', 'kurtosis_f_w5', 'kurtosis_f_w6',
       'kurtosis_f_w7', 'kurtosis_f_w8', 'kurtosis_f_w9', 'kurtosis_f_w10',
       'kurtosis_f_w11', 'kurtosis_f_w12'],
      dtype='object', length=573)

In [102]:
data.drop(columns=["Unnamed: 0"], inplace=True)

In [235]:
data.head(30)

,session,participant,gesture,index,iemg,mav,ssi,myopulse,wflen,diffvar,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,1,1,10,0,11996.863087,0.041842,997.135823,6.224512,8143.485828,0.001888,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,1,1,10,0,10550.150337,0.036796,1019.627142,6.443555,6646.721181,0.001438,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,1,1,10,0,9230.366582,0.032193,630.013233,6.033789,6023.438546,0.001169,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,1,1,10,0,15949.504704,0.055627,1995.234560,4.999609,9318.559036,0.003030,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,1,1,10,0,15936.956936,0.055584,1830.412922,5.870117,10476.465579,0.003688,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713
5,1,1,10,0,10302.082204,0.035931,957.747699,5.384570,6660.235040,0.002048,...,194.169246,160.867419,127.623463,144.913630,157.298899,94.204471,155.422302,54.005036,53.646785,61.060966
6,1,1,10,0,19771.041940,0.068956,2959.381173,5.280957,11976.937109,0.005174,...,113.893317,54.651356,46.359047,96.793045,54.653056,182.248943,65.594571,81.105593,66.837683,61.510619
7,1,1,11,0,28104.586469,0.098021,7096.757592,4.965234,12613.872878,0.006058,...,140.633258,44.404009,68.097183,49.395457,147.219607,45.064573,157.636975,59.939376,66.865776,91.715022
8,1,1,11,0,27253.345839,0.095052,6467.184093,4.996094,12258.059044,0.005827,...,52.195250,43.058702,29.454097,74.433499,48.285962,29.769905,114.525041,80.717165,52.239566,94.334606
9,1,1,11,0,24522.650663,0.085528,4589.346795,4.919336,11242.445344,0.004575,...,38.664579,39.118622,30.194664,28.584404,38.329797,46.371623,34.716442,31.226083,49.508191,34.297186


In [104]:
features = data.iloc[:, 4:]

In [105]:
features

,iemg,mav,ssi,myopulse,wflen,diffvar,dasd,willison,power_f1,power_f2,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,11996.863087,0.041842,997.135823,6.224512,8143.485828,0.001888,0.034480,10277,0.002297,0.004143,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,10550.150337,0.036796,1019.627142,6.443555,6646.721181,0.001438,0.030668,6420,0.003276,0.005005,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,9230.366582,0.032193,630.013233,6.033789,6023.438546,0.001169,0.027090,5052,0.001570,0.002424,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,15949.504704,0.055627,1995.234560,4.999609,9318.559036,0.003030,0.045251,19492,0.004655,0.006518,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,15936.956936,0.055584,1830.412922,5.870117,10476.465579,0.003688,0.047223,21471,0.003665,0.005650,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,7435.557234,0.025933,369.818221,6.280762,5792.222983,0.001070,0.026210,5307,0.001265,0.000794,...,29.589425,25.460694,182.962449,34.957161,18.846525,37.714119,26.569683,32.614845,12.793219,16.657922
15347,10471.625678,0.036522,713.052200,6.967187,8628.380136,0.002070,0.035779,11624,0.002672,0.002797,...,18.226247,35.715660,113.822913,64.223311,8.214120,39.211218,29.910432,26.426457,24.892159,82.241098
15348,11279.138159,0.039339,815.440185,6.917773,9229.795138,0.002355,0.038498,14370,0.002550,0.002242,...,15.486525,29.417342,25.153035,19.632020,15.590478,15.462391,34.940941,37.564280,14.208021,15.811267
15349,9210.946156,0.032125,563.422838,7.359570,8040.357766,0.001839,0.033699,10798,0.001746,0.001153,...,14.527208,20.281059,20.418717,18.471741,14.276295,10.083487,27.337412,12.804675,20.220452,37.494222


In [106]:
labels = data.iloc[:, 2] - 1

In [107]:
labels.unique()


array([ 9, 10, 11, 12, 13, 14, 15, 16,  0,  1,  2,  3,  4,  5,  6,  7,  8],
      dtype=int64)

In [108]:
labels

0        9
1        9
2        9
3        9
4        9
        ..
15346    8
15347    8
15348    8
15349    8
15350    8
Name: gesture, Length: 15351, dtype: int64

In [109]:
features

,iemg,mav,ssi,myopulse,wflen,diffvar,dasd,willison,power_f1,power_f2,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,11996.863087,0.041842,997.135823,6.224512,8143.485828,0.001888,0.034480,10277,0.002297,0.004143,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,10550.150337,0.036796,1019.627142,6.443555,6646.721181,0.001438,0.030668,6420,0.003276,0.005005,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,9230.366582,0.032193,630.013233,6.033789,6023.438546,0.001169,0.027090,5052,0.001570,0.002424,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,15949.504704,0.055627,1995.234560,4.999609,9318.559036,0.003030,0.045251,19492,0.004655,0.006518,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,15936.956936,0.055584,1830.412922,5.870117,10476.465579,0.003688,0.047223,21471,0.003665,0.005650,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,7435.557234,0.025933,369.818221,6.280762,5792.222983,0.001070,0.026210,5307,0.001265,0.000794,...,29.589425,25.460694,182.962449,34.957161,18.846525,37.714119,26.569683,32.614845,12.793219,16.657922
15347,10471.625678,0.036522,713.052200,6.967187,8628.380136,0.002070,0.035779,11624,0.002672,0.002797,...,18.226247,35.715660,113.822913,64.223311,8.214120,39.211218,29.910432,26.426457,24.892159,82.241098
15348,11279.138159,0.039339,815.440185,6.917773,9229.795138,0.002355,0.038498,14370,0.002550,0.002242,...,15.486525,29.417342,25.153035,19.632020,15.590478,15.462391,34.940941,37.564280,14.208021,15.811267
15349,9210.946156,0.032125,563.422838,7.359570,8040.357766,0.001839,0.033699,10798,0.001746,0.001153,...,14.527208,20.281059,20.418717,18.471741,14.276295,10.083487,27.337412,12.804675,20.220452,37.494222


In [115]:
scaler = StandardScaler()

X = scaler.fit_transform(features)
Y = labels.values

In [116]:
Y.shape

(15351,)

In [117]:
X.shape

(15351, 568)

In [118]:
X

array([[ 0.02392332,  0.02392332, -0.1874717 , ..., -0.04878718,
        -0.11424713,  0.38753344],
       [-0.18785798, -0.18785798, -0.17863791, ...,  1.18393595,
         2.47507057,  3.60160943],
       [-0.38105841, -0.38105841, -0.3316644 , ...,  0.796314  ,
         1.05833522,  0.36438855],
       ...,
       [-0.08114296, -0.08114296, -0.25883529, ..., -0.22266456,
        -0.40618315, -0.40441409],
       [-0.38390132, -0.38390132, -0.35781874, ..., -0.44472878,
        -0.34445757, -0.2378717 ],
       [-0.33314875, -0.33314875, -0.33456336, ..., -0.3679266 ,
        -0.42503833, -0.2247458 ]])

In [185]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=43)

In [186]:
x_train

array([[-0.11629552, -0.11629552, -0.19289876, ..., -0.19079338,
        -0.21617383, -0.23099812],
       [ 3.45405924,  3.45405924,  3.97590219, ..., -0.3774615 ,
        -0.37834971, -0.37972356],
       [ 0.19801118,  0.19801118, -0.11286955, ..., -0.28453885,
        -0.32990027, -0.21279221],
       ...,
       [ 0.09244361,  0.09244361, -0.10603224, ...,  0.8706434 ,
        -0.03801353,  0.09171287],
       [-0.19932551, -0.19932551, -0.29204992, ..., -0.3768488 ,
        -0.17473026, -0.38905995],
       [-0.30814251, -0.30814251, -0.30755708, ..., -0.01050093,
        -0.23205439, -0.20590404]])

In [187]:
x_test

array([[-0.27462652, -0.27462652, -0.3221948 , ..., -0.45198375,
        -0.31674569, -0.41387085],
       [-0.1133239 , -0.1133239 , -0.24361123, ...,  0.32942267,
        -0.21838578, -0.27355456],
       [ 2.71427156,  2.71427156,  2.38994258, ..., -0.15937449,
         0.22752784,  0.39048761],
       ...,
       [ 0.83047173,  0.83047173,  0.28591516, ...,  0.01952809,
        -0.26177294, -0.23809514],
       [-0.18717363, -0.18717363, -0.28588523, ..., -0.31540792,
        -0.35467747, -0.33105603],
       [-0.13234052, -0.13234052, -0.21027761, ...,  0.64867224,
        -0.18352401,  0.04790248]])

In [188]:
# x_train = x_train.values
# x_test = x_test.values

# y_train = y_train.values
# y_test = y_test.values

In [189]:
y_test


array([3, 1, 6, ..., 6, 2, 3], dtype=int64)

In [206]:
def simple_model(input_shape, output_shape):
    model = nn.Sequential(
        nn.Linear(input_shape, 32),  # Input layer: Fully connected (linear) with 64 units
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(32, 64),
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(64, output_shape)  # Output layer: Fully connected (linear) with 'output_shape' units
    )
    
    return model

In [207]:
train_dataset = TensorDataset(torch.tensor(x_train).type(torch.float32), torch.tensor(y_train).type(torch.LongTensor))
test_dataset = TensorDataset(torch.tensor(x_test).type(torch.float32), torch.tensor(y_test).type(torch.LongTensor))

In [208]:
len(np.unique(y_train))

17

In [220]:
# Define batch size and whether to shuffle the data
batch_size = 128
shuffle = True

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [221]:
len(train_dataset)

12280

In [222]:
x_train.shape

(12280, 568)

In [223]:
model = simple_model(x_train.shape[1], len(np.unique(y_train)))

In [224]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [225]:
# Define training parameters
num_epochs = 50

In [226]:
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and log metrics
    train_loss = total_train_loss / len(train_loader)
    val_loss = total_val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch [1/50] - Train Loss: 1.4938, Val Loss: 1.0819, Val Accuracy: 63.79%
Epoch [2/50] - Train Loss: 0.8986, Val Loss: 0.9366, Val Accuracy: 69.16%
Epoch [3/50] - Train Loss: 0.7881, Val Loss: 0.8759, Val Accuracy: 72.45%
Epoch [4/50] - Train Loss: 0.7137, Val Loss: 0.8795, Val Accuracy: 73.66%
Epoch [5/50] - Train Loss: 0.6595, Val Loss: 0.8652, Val Accuracy: 74.31%
Epoch [6/50] - Train Loss: 0.6511, Val Loss: 0.8050, Val Accuracy: 75.87%
Epoch [7/50] - Train Loss: 0.6010, Val Loss: 0.9471, Val Accuracy: 73.10%
Epoch [8/50] - Train Loss: 0.6138, Val Loss: 0.8277, Val Accuracy: 75.55%
Epoch [9/50] - Train Loss: 0.5269, Val Loss: 0.8251, Val Accuracy: 77.53%
Epoch [10/50] - Train Loss: 0.4757, Val Loss: 0.8799, Val Accuracy: 77.89%
Epoch [11/50] - Train Loss: 0.4662, Val Loss: 0.8168, Val Accuracy: 78.35%
Epoch [12/50] - Train Loss: 0.4685, Val Loss: 1.0161, Val Accuracy: 75.71%
Epoch [13/50] - Train Loss: 0.4553, Val Loss: 0.9261, Val Accuracy: 77.21%
Epoch [14/50] - Train Loss: 0.4248

In [231]:
def predict(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    y_pred = []

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.tolist())

    return y_pred

In [232]:
y_test_pred = predict(model, test_loader)

In [233]:
y_test_pred

[3,
 1,
 6,
 1,
 2,
 14,
 6,
 4,
 11,
 8,
 7,
 15,
 16,
 1,
 7,
 14,
 15,
 9,
 16,
 2,
 2,
 4,
 8,
 2,
 10,
 13,
 0,
 3,
 5,
 4,
 11,
 5,
 7,
 5,
 11,
 8,
 13,
 4,
 9,
 16,
 6,
 15,
 4,
 7,
 16,
 15,
 0,
 9,
 5,
 15,
 9,
 7,
 2,
 14,
 3,
 9,
 6,
 2,
 3,
 8,
 7,
 11,
 10,
 4,
 1,
 13,
 3,
 12,
 9,
 6,
 9,
 6,
 4,
 13,
 5,
 16,
 9,
 10,
 7,
 4,
 16,
 13,
 11,
 10,
 6,
 1,
 4,
 1,
 13,
 9,
 7,
 6,
 7,
 14,
 12,
 1,
 6,
 6,
 2,
 3,
 16,
 15,
 15,
 8,
 5,
 5,
 4,
 3,
 7,
 3,
 8,
 8,
 8,
 2,
 3,
 16,
 5,
 16,
 4,
 8,
 3,
 15,
 13,
 7,
 13,
 10,
 8,
 7,
 9,
 2,
 10,
 12,
 6,
 9,
 8,
 9,
 15,
 6,
 12,
 0,
 11,
 13,
 7,
 15,
 11,
 14,
 12,
 1,
 12,
 8,
 2,
 8,
 12,
 6,
 13,
 0,
 8,
 10,
 10,
 16,
 14,
 15,
 3,
 14,
 10,
 8,
 15,
 13,
 8,
 14,
 5,
 4,
 0,
 16,
 0,
 13,
 9,
 0,
 15,
 7,
 5,
 1,
 7,
 11,
 7,
 11,
 2,
 15,
 1,
 1,
 2,
 16,
 5,
 4,
 5,
 15,
 14,
 1,
 2,
 12,
 16,
 5,
 10,
 10,
 11,
 8,
 2,
 7,
 7,
 9,
 15,
 1,
 12,
 1,
 11,
 6,
 14,
 16,
 10,
 0,
 14,
 12,
 10,
 6,
 11,
 1,
 10,
 1,

In [234]:
report = classification_report(y_test, y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.63      0.66      0.65       159
           1       0.66      0.68      0.67       165
           2       0.72      0.78      0.75       180
           3       0.76      0.66      0.70       204
           4       0.85      0.82      0.84       186
           5       0.91      0.87      0.89       180
           6       0.68      0.73      0.70       184
           7       0.81      0.85      0.83       170
           8       0.85      0.70      0.77       192
           9       0.78      0.84      0.81       207
          10       0.99      0.95      0.97       167
          11       0.94      0.90      0.92       191
          12       0.90      0.91      0.91       149
          13       0.89      0.90      0.89       192
          14       0.88      0.91      0.89       177
          15       0.73      0.71      0.72       191
          16       0.90      0.97      0.93       177

    accuracy              

## General split

In [389]:
data = pd.read_csv('../data/gesture-recognition-and-biometrics-electromyogram-grabmyo-1.0.2/features_split.csv')

In [390]:
data.head()

,Unnamed: 0,session,participant,gesture,index,iemg,mav,ssi,myopulse,wflen,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,0,1,1,10,0,11996.863087,0.041842,997.135823,6.224512,8143.485828,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,1,1,1,10,0,10550.150337,0.036796,1019.627142,6.443555,6646.721181,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,2,1,1,10,0,9230.366582,0.032193,630.013233,6.033789,6023.438546,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,3,1,1,10,0,15949.504704,0.055627,1995.234560,4.999609,9318.559036,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,4,1,1,10,0,15936.956936,0.055584,1830.412922,5.870117,10476.465579,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713


In [391]:
data.columns

Index(['Unnamed: 0', 'session', 'participant', 'gesture', 'index', 'iemg',
       'mav', 'ssi', 'myopulse', 'wflen',
       ...
       'kurtosis_f_w3', 'kurtosis_f_w4', 'kurtosis_f_w5', 'kurtosis_f_w6',
       'kurtosis_f_w7', 'kurtosis_f_w8', 'kurtosis_f_w9', 'kurtosis_f_w10',
       'kurtosis_f_w11', 'kurtosis_f_w12'],
      dtype='object', length=573)

In [392]:
data.drop(columns=["Unnamed: 0"], inplace=True)

In [393]:
X = data.iloc[:, 4:].values
Y = (data.iloc[:, 2]-1).values

In [394]:
# X = scaler.fit_transform(X)

In [395]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=43)

In [396]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [397]:
def simple_model(input_shape, output_shape):
    model = nn.Sequential(
        nn.Linear(input_shape, 32),  # Input layer: Fully connected (linear) with 64 units
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(32, 64),
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(64, output_shape)  # Output layer: Fully connected (linear) with 'output_shape' units
    )
    
    return model

In [398]:
train_dataset = TensorDataset(torch.tensor(x_train).type(torch.float32), torch.tensor(y_train).type(torch.LongTensor))
test_dataset = TensorDataset(torch.tensor(x_test).type(torch.float32), torch.tensor(y_test).type(torch.LongTensor))

In [399]:
len(np.unique(y_train))

17

In [400]:
# Define batch size and whether to shuffle the data
batch_size = 128
shuffle = True

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [380]:
len(train_dataset)

12280

In [381]:
x_train.shape

(12280, 568)

In [420]:
model = simple_model(x_train.shape[1], len(np.unique(y_train)))

In [421]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=5e-4)

In [422]:
# Define training parameters
num_epochs = 100

In [423]:
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and log metrics
    train_loss = total_train_loss / len(train_loader)
    val_loss = total_val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch [1/100] - Train Loss: 2.5523, Val Loss: 22793884.8442, Val Accuracy: 28.82%
Epoch [2/100] - Train Loss: 1.9588, Val Loss: 89489548.2231, Val Accuracy: 47.38%
Epoch [3/100] - Train Loss: 1.4388, Val Loss: 146670934.5514, Val Accuracy: 58.87%
Epoch [4/100] - Train Loss: 1.1389, Val Loss: 186222422.3765, Val Accuracy: 64.34%
Epoch [5/100] - Train Loss: 0.9826, Val Loss: 241710395.6160, Val Accuracy: 66.56%
Epoch [6/100] - Train Loss: 0.8839, Val Loss: 262570496.8720, Val Accuracy: 69.65%
Epoch [7/100] - Train Loss: 0.8154, Val Loss: 273289552.8364, Val Accuracy: 70.99%
Epoch [8/100] - Train Loss: 0.7639, Val Loss: 271160811.4649, Val Accuracy: 71.38%
Epoch [9/100] - Train Loss: 0.7216, Val Loss: 353456683.4313, Val Accuracy: 72.78%
Epoch [10/100] - Train Loss: 0.6857, Val Loss: 316731232.7453, Val Accuracy: 74.11%
Epoch [11/100] - Train Loss: 0.6555, Val Loss: 337435360.7227, Val Accuracy: 74.41%
Epoch [12/100] - Train Loss: 0.6300, Val Loss: 371203776.6950, Val Accuracy: 75.64%
Epo

Epoch [99/100] - Train Loss: 0.1605, Val Loss: 2465930752.7656, Val Accuracy: 82.06%
Epoch [100/100] - Train Loss: 0.1619, Val Loss: 2642004992.7620, Val Accuracy: 81.86%


In [424]:
def predict(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    y_pred = []

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.tolist())

    return y_pred

In [425]:
y_test_pred = predict(model, test_loader)

In [426]:
report = classification_report(y_test, y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.69      0.67      0.68       159
           1       0.68      0.72      0.70       165
           2       0.77      0.83      0.80       180
           3       0.72      0.72      0.72       204
           4       0.86      0.78      0.82       186
           5       0.88      0.91      0.89       180
           6       0.73      0.67      0.70       184
           7       0.80      0.88      0.84       170
           8       0.79      0.73      0.76       192
           9       0.83      0.80      0.82       207
          10       0.98      0.97      0.97       167
          11       0.90      0.91      0.90       191
          12       0.81      0.91      0.86       149
          13       0.92      0.89      0.90       192
          14       0.84      0.88      0.86       177
          15       0.76      0.75      0.76       191
          16       0.94      0.94      0.94       177

    accuracy              

## Split on participants

In [489]:
participants = list(range(5,43))
test_participants = list(range(1,5))

In [492]:
train_df = data[data['participant'].isin(participants)]
test_df = data[data['participant'].isin(test_participants)]

In [493]:
train_df

,session,participant,gesture,index,iemg,mav,ssi,myopulse,wflen,diffvar,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
119,1,10,10,0,6797.436339,0.023708,313.982240,7.401855,5733.301325,0.000904,...,79.115045,32.591676,58.994440,54.839028,50.292229,104.307591,61.155945,196.373417,92.713179,405.962532
120,1,10,10,0,14079.062764,0.049104,1291.830907,6.844043,10385.642760,0.002613,...,12.381097,12.099599,10.995862,17.486544,14.211067,14.489695,16.576432,160.712336,79.746286,170.230168
121,1,10,10,0,11472.717905,0.040014,960.778273,6.713477,8706.221451,0.002182,...,48.730862,305.459894,212.763290,36.980016,29.056499,299.450092,55.901194,64.921149,59.408569,167.539726
122,1,10,10,0,17507.830172,0.061062,2332.784349,6.788477,13368.566593,0.005666,...,51.993926,51.528948,20.339587,43.117535,34.757809,59.816264,20.777295,59.169183,45.939629,43.873509
123,1,10,10,0,18614.528512,0.064922,2580.763916,6.342676,13226.563635,0.005325,...,306.354173,55.281935,15.773219,26.464685,84.562723,76.696436,19.323875,37.940924,24.052192,41.660051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,3,9,9,0,7435.557234,0.025933,369.818221,6.280762,5792.222983,0.001070,...,29.589425,25.460694,182.962449,34.957161,18.846525,37.714119,26.569683,32.614845,12.793219,16.657922
15347,3,9,9,0,10471.625678,0.036522,713.052200,6.967187,8628.380136,0.002070,...,18.226247,35.715660,113.822913,64.223311,8.214120,39.211218,29.910432,26.426457,24.892159,82.241098
15348,3,9,9,0,11279.138159,0.039339,815.440185,6.917773,9229.795138,0.002355,...,15.486525,29.417342,25.153035,19.632020,15.590478,15.462391,34.940941,37.564280,14.208021,15.811267
15349,3,9,9,0,9210.946156,0.032125,563.422838,7.359570,8040.357766,0.001839,...,14.527208,20.281059,20.418717,18.471741,14.276295,10.083487,27.337412,12.804675,20.220452,37.494222


In [494]:
x_train = train_df.iloc[:, 4:].values
y_train = (train_df.iloc[:, 2] - 1).values

x_test = test_df.iloc[:, 4:].values
y_test = (test_df.iloc[:, 2] - 1).values


In [495]:
scaler = StandardScaler()

In [496]:
scaler.fit(x_train)

StandardScaler()

In [497]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [498]:
x_train

array([[-0.71151404, -0.71151404, -0.44104706, ...,  1.18273398,
         0.3947441 ,  2.51393605],
       [ 0.3502168 ,  0.3502168 , -0.06034413, ...,  0.86820923,
         0.26368513,  0.75742463],
       [-0.02981323, -0.02981323, -0.18923188, ...,  0.02334703,
         0.05812778,  0.73737735],
       ...,
       [-0.05803901, -0.05803901, -0.24581592, ..., -0.21793596,
        -0.39872317, -0.39319618],
       [-0.35960121, -0.35960121, -0.34393309, ..., -0.43631153,
        -0.33795433, -0.23163008],
       [-0.30904915, -0.30904915, -0.32088124, ..., -0.36078509,
        -0.41728609, -0.21889639]])

In [499]:
x_test.shape

(1428, 568)

In [500]:
x_train.shape

(13566, 568)

In [501]:
def simple_model(input_shape, output_shape):
    model = nn.Sequential(
        nn.Linear(input_shape, 32),  # Input layer: Fully connected (linear) with 64 units
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(32, 64),
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(64, output_shape)  # Output layer: Fully connected (linear) with 'output_shape' units
    )
    
    return model

In [502]:
train_dataset = TensorDataset(torch.tensor(x_train).type(torch.float32), torch.tensor(y_train).type(torch.LongTensor))
test_dataset = TensorDataset(torch.tensor(x_test).type(torch.float32), torch.tensor(y_test).type(torch.LongTensor))

In [503]:
len(np.unique(y_train))

17

In [504]:
# Define batch size and whether to shuffle the data
batch_size = 128
shuffle = True

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [505]:
model = simple_model(x_train.shape[1], len(np.unique(y_train)))

In [506]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [507]:
# Define training parameters
num_epochs = 100

In [508]:
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and log metrics
    train_loss = total_train_loss / len(train_loader)
    val_loss = total_val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch [1/100] - Train Loss: 2.2616, Val Loss: 1.7581, Val Accuracy: 46.85%
Epoch [2/100] - Train Loss: 1.3030, Val Loss: 1.4237, Val Accuracy: 58.12%
Epoch [3/100] - Train Loss: 0.9410, Val Loss: 1.3434, Val Accuracy: 58.33%
Epoch [4/100] - Train Loss: 0.7860, Val Loss: 1.3820, Val Accuracy: 58.75%
Epoch [5/100] - Train Loss: 0.6940, Val Loss: 1.3767, Val Accuracy: 60.78%
Epoch [6/100] - Train Loss: 0.6312, Val Loss: 1.3106, Val Accuracy: 61.06%
Epoch [7/100] - Train Loss: 0.5876, Val Loss: 1.4357, Val Accuracy: 60.85%
Epoch [8/100] - Train Loss: 0.5474, Val Loss: 1.4316, Val Accuracy: 61.97%
Epoch [9/100] - Train Loss: 0.5167, Val Loss: 1.5336, Val Accuracy: 59.73%
Epoch [10/100] - Train Loss: 0.4869, Val Loss: 1.5775, Val Accuracy: 59.24%
Epoch [11/100] - Train Loss: 0.4674, Val Loss: 1.5543, Val Accuracy: 62.25%
Epoch [12/100] - Train Loss: 0.4422, Val Loss: 1.6126, Val Accuracy: 62.11%
Epoch [13/100] - Train Loss: 0.4295, Val Loss: 1.6357, Val Accuracy: 61.27%
Epoch [14/100] - Trai

In [509]:
y_test_pred = predict(model, test_loader)

In [510]:
y_test_pred

[9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 12,
 13,
 13,
 13,
 13,
 14,
 9,
 14,
 14,
 14,
 14,
 14,
 4,
 15,
 15,
 0,
 15,
 15,
 15,
 16,
 12,
 16,
 16,
 16,
 16,
 16,
 0,
 0,
 15,
 0,
 0,
 15,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 12,
 1,
 2,
 2,
 2,
 6,
 6,
 2,
 6,
 12,
 12,
 1,
 6,
 8,
 0,
 8,
 4,
 8,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 12,
 6,
 6,
 6,
 7,
 7,
 7,
 5,
 5,
 7,
 5,
 8,
 0,
 8,
 0,
 8,
 8,
 8,
 9,
 9,
 12,
 9,
 0,
 9,
 12,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 12,
 12,
 11,
 11,
 11,
 12,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 12,
 12,
 12,
 4,
 14,
 14,
 4,
 14,
 14,
 4,
 1,
 15,
 0,
 0,
 15,
 15,
 15,
 12,
 16,
 13,
 13,
 13,
 12,
 13,
 0,
 0,
 16,
 0,
 15,
 15,
 15,
 2,
 1,
 1,
 1,
 1,
 12,
 12,
 2,
 2,
 10,
 2,
 2,
 2,
 2,
 13,
 13,
 13,
 13,
 7,
 7,
 13,
 8,
 11,
 4,
 4,
 4,
 11,
 4,
 5,
 1,
 1,
 13,
 1,
 1,
 5,
 2,
 2,
 2,
 2,
 2,


In [511]:
y_test

array([9, 9, 9, ..., 8, 8, 8], dtype=int64)

In [512]:
report = classification_report(y_test, y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.53      0.61      0.57        84
           1       0.49      0.43      0.46        84
           2       0.52      0.57      0.55        84
           3       0.30      0.27      0.29        84
           4       0.72      0.50      0.59        84
           5       0.79      0.60      0.68        84
           6       0.37      0.55      0.44        84
           7       0.64      0.40      0.50        84
           8       0.69      0.52      0.59        84
           9       0.79      0.45      0.58        84
          10       0.83      0.89      0.86        84
          11       0.88      0.93      0.90        84
          12       0.47      0.74      0.58        84
          13       0.66      0.74      0.70        84
          14       0.52      0.79      0.63        84
          15       0.66      0.46      0.55        84
          16       0.88      0.92      0.90        84

    accuracy              

### Split by session

In [467]:
participants = [1,3]
test_participants = [2]

In [468]:
train_df = data[data['session'].isin(participants)]
test_df = data[data['session'].isin(test_participants)]

In [469]:
train_df

,session,participant,gesture,index,iemg,mav,ssi,myopulse,wflen,diffvar,...,kurtosis_f_w3,kurtosis_f_w4,kurtosis_f_w5,kurtosis_f_w6,kurtosis_f_w7,kurtosis_f_w8,kurtosis_f_w9,kurtosis_f_w10,kurtosis_f_w11,kurtosis_f_w12
0,1,1,10,0,11996.863087,0.041842,997.135823,6.224512,8143.485828,0.001888,...,61.940683,116.130710,31.371157,107.176549,64.459302,241.232060,54.880235,56.951175,42.644289,118.918727
1,1,1,10,0,10550.150337,0.036796,1019.627142,6.443555,6646.721181,0.001438,...,160.171639,87.566832,154.616093,280.595181,160.756112,98.589493,135.655396,194.396727,294.858908,537.374741
2,1,1,10,0,9230.366582,0.032193,630.013233,6.033789,6023.438546,0.001169,...,195.666850,90.568034,94.299202,276.105575,101.698579,49.607516,150.348389,151.177847,156.860637,115.905382
3,1,1,10,0,15949.504704,0.055627,1995.234560,4.999609,9318.559036,0.003030,...,33.886272,80.377706,46.373047,64.445165,31.711430,33.328942,50.491362,81.876212,69.070232,142.723324
4,1,1,10,0,15936.956936,0.055584,1830.412922,5.870117,10476.465579,0.003688,...,48.630658,39.722917,25.079065,84.877336,29.229362,48.843301,44.917769,39.354396,78.705208,176.820713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,3,9,9,0,7435.557234,0.025933,369.818221,6.280762,5792.222983,0.001070,...,29.589425,25.460694,182.962449,34.957161,18.846525,37.714119,26.569683,32.614845,12.793219,16.657922
15347,3,9,9,0,10471.625678,0.036522,713.052200,6.967187,8628.380136,0.002070,...,18.226247,35.715660,113.822913,64.223311,8.214120,39.211218,29.910432,26.426457,24.892159,82.241098
15348,3,9,9,0,11279.138159,0.039339,815.440185,6.917773,9229.795138,0.002355,...,15.486525,29.417342,25.153035,19.632020,15.590478,15.462391,34.940941,37.564280,14.208021,15.811267
15349,3,9,9,0,9210.946156,0.032125,563.422838,7.359570,8040.357766,0.001839,...,14.527208,20.281059,20.418717,18.471741,14.276295,10.083487,27.337412,12.804675,20.220452,37.494222


In [470]:
x_train = train_df.iloc[:, 4:].values
y_train = (train_df.iloc[:, 2] - 1).values

x_test = test_df.iloc[:, 4:].values
y_test = (test_df.iloc[:, 2] - 1).values


In [471]:
scaler = StandardScaler()

In [472]:
scaler.fit(x_train)

StandardScaler()

In [473]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [474]:
def simple_model(input_shape, output_shape):
    model = nn.Sequential(
        nn.Linear(input_shape, 32),  # Input layer: Fully connected (linear) with 64 units
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(32, 64),
        nn.ReLU(),  # Activation function: ReLU
        nn.Linear(64, output_shape)  # Output layer: Fully connected (linear) with 'output_shape' units
    )
    
    return model

In [475]:
train_dataset = TensorDataset(torch.tensor(x_train).type(torch.float32), torch.tensor(y_train).type(torch.LongTensor))
test_dataset = TensorDataset(torch.tensor(x_test).type(torch.float32), torch.tensor(y_test).type(torch.LongTensor))

In [476]:
len(np.unique(y_train))

17

In [477]:
# Define batch size and whether to shuffle the data
batch_size = 128
shuffle = True

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [478]:
model = simple_model(x_train.shape[1], len(np.unique(y_train)))

In [479]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [480]:
# Define training parameters
num_epochs = 100

In [481]:
# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_train_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and log metrics
    train_loss = total_train_loss / len(train_loader)
    val_loss = total_val_loss / len(test_loader)
    val_accuracy = 100 * correct / total

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch [1/100] - Train Loss: 2.4191, Val Loss: 1365127.5563, Val Accuracy: 38.38%
Epoch [2/100] - Train Loss: 1.5826, Val Loss: 9924327.7019, Val Accuracy: 58.84%
Epoch [3/100] - Train Loss: 1.1271, Val Loss: 21853153.0808, Val Accuracy: 65.88%
Epoch [4/100] - Train Loss: 0.9310, Val Loss: 27828983.3690, Val Accuracy: 69.55%
Epoch [5/100] - Train Loss: 0.8197, Val Loss: 34200506.5026, Val Accuracy: 72.11%
Epoch [6/100] - Train Loss: 0.7383, Val Loss: 36717312.8797, Val Accuracy: 72.52%
Epoch [7/100] - Train Loss: 0.6748, Val Loss: 37467072.8526, Val Accuracy: 73.99%
Epoch [8/100] - Train Loss: 0.6270, Val Loss: 47859888.8476, Val Accuracy: 74.42%
Epoch [9/100] - Train Loss: 0.5921, Val Loss: 45645607.2486, Val Accuracy: 74.52%
Epoch [10/100] - Train Loss: 0.5652, Val Loss: 42729972.0853, Val Accuracy: 75.20%
Epoch [11/100] - Train Loss: 0.5350, Val Loss: 38915735.2525, Val Accuracy: 75.83%
Epoch [12/100] - Train Loss: 0.5119, Val Loss: 47672640.8599, Val Accuracy: 77.14%
Epoch [13/100] 

Epoch [100/100] - Train Loss: 0.0618, Val Loss: 299416731.6430, Val Accuracy: 76.76%


In [486]:
y_test_pred = predict(model, test_loader)

In [487]:
y_test

array([9, 9, 9, ..., 8, 8, 8], dtype=int64)

In [488]:
report = classification_report(y_test, y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.65      0.61      0.63       301
           1       0.65      0.51      0.57       301
           2       0.69      0.72      0.71       301
           3       0.57      0.64      0.61       301
           4       0.74      0.80      0.77       301
           5       0.88      0.87      0.87       301
           6       0.66      0.65      0.66       301
           7       0.78      0.85      0.81       301
           8       0.73      0.72      0.72       301
           9       0.79      0.82      0.80       301
          10       0.95      0.92      0.93       301
          11       0.86      0.85      0.86       301
          12       0.81      0.82      0.82       301
          13       0.84      0.83      0.84       301
          14       0.82      0.79      0.81       301
          15       0.70      0.66      0.68       301
          16       0.93      0.96      0.94       301

    accuracy              